In [1]:
import os
import json
from datasets import load_dataset
from PIL import Image


# Путь к выходной директории
output_dir = "yolov5/dataset"
data_dir = os.path.join(output_dir, "custom_data")
os.makedirs(data_dir, exist_ok=True)

# Маппинг классов
class_mapping = {
    "menu.nm": 0,
    "menu.price": 1,
    "menu.cnt": 2,
    "total.total_price": 3,
}

# Функция для вычисления bounding box
def get_bbox_info(quad):
    x_coords = [quad[f"x{i}"] for i in range(1, 5)]
    y_coords = [quad[f"y{i}"] for i in range(1, 5)]

    min_x, max_x = min(x_coords), max(x_coords)
    min_y, max_y = min(y_coords), max(y_coords)

    center_x = min_x + (max_x - min_x) / 2
    center_y = min_y + (max_y - min_y) / 2
    width = max_x - min_x
    height = max_y - min_y

    return center_x, center_y, width, height


# Функция сохранения аннотаций
def save_annotation(example, image_id, output_folder):
    filename = f"{image_id}.txt"
    path = os.path.join(output_folder, filename)

    try:
        gt_dict = json.loads(example["ground_truth"])
    except json.JSONDecodeError:
        return

    # Получаем размер изображения: сначала из мета, иначе из PIL.Image
    meta = gt_dict.get("meta", {})
    img_size = meta.get("image_size")
    if img_size:
        img_width = img_size["width"]
        img_height = img_size["height"]
    else:
        try:
            img_width, img_height = example["image"].size
        except Exception as e:
            print(f"⚠️ Не удалось получить размер изображения для {image_id}, используем 1000x1000")
            img_width = img_height = 1000

    with open(path, 'w') as f:
        for item in gt_dict["valid_line"]:
            category = item["category"]
            words = item["words"]

            cls_id = class_mapping.get(category)
            if cls_id is None:
                continue  # пропускаем неизвестные или исключённые классы

            for word in words:
                quad = word["quad"]
                cx, cy, w, h = get_bbox_info(quad)

                # Нормализация относительно реального размера изображения
                cx_norm = cx / img_width
                cy_norm = cy / img_height
                w_norm = w / img_width
                h_norm = h / img_height

                line = f"{cls_id} {cx_norm:.10f} {cy_norm:.10f} {w_norm:.10f} {h_norm:.10f}\n"
                f.write(line)


# Функция сохранения изображения (принимает PIL.Image)
def save_image(image_obj, image_id, output_folder):
    try:
        image_obj.convert("RGB").save(os.path.join(output_folder, f"{image_id}.jpg"), "JPEG")
    except Exception as e:
        print(f"❌ Ошибка при сохранении изображения {image_id}: {e}")


# Загрузка датасета
dataset = load_dataset("naver-clova-ix/cord-v2", trust_remote_code=True)

# Списки путей
paths = {"train": [], "validation": [], "test": []}
global_idx = 1  # Общий счётчик изображений

# Обработка всех сплитов подряд
for split in ["train", "validation", "test"]:
    print(f"Processing {split} split...")
    for example in dataset[split]:
        image_id = f"image{global_idx:03d}"

        # Парсим ground_truth
        try:
            gt_dict = json.loads(example["ground_truth"])
        except json.JSONDecodeError:
            print(f"⚠️ Пропущено изображение {image_id}: ошибка парсинга ground_truth")
            global_idx += 1
            continue

        # Сохраняем изображение
        try:
            save_image(example["image"], image_id, data_dir)
        except Exception as e:
            print(f"❌ Не удалось сохранить изображение {image_id}: {e}")
            global_idx += 1
            continue

        # Сохраняем аннотацию
        save_annotation(example, image_id, data_dir)

        # Добавляем путь к изображению
        paths[split].append(f"dataset/custom_data/{image_id}.jpg")

        # Переходим к следующему номеру
        global_idx += 1


# Сохраняем пути в файлы
with open(os.path.join(output_dir, "custom_train.txt"), "w") as f:
    f.write("\n".join(paths["train"]))

with open(os.path.join(output_dir, "custom_valid.txt"), "w") as f:
    f.write("\n".join(paths["validation"]))

with open(os.path.join(output_dir, "custom_train_test.txt"), "w") as f:
    f.write("\n".join(paths["test"]))

print(f"\n✅ Готово! Обработано изображений: {global_idx - 1}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

(…)-00000-of-00004-b4aaeceff1d90ecb.parquet:   0%|          | 0.00/490M [00:00<?, ?B/s]

(…)-00001-of-00004-7dbbe248962764c5.parquet:   0%|          | 0.00/441M [00:00<?, ?B/s]

...

(…)-00000-of-00001-9c204eb3f4e11791.parquet:   0%|          | 0.00/234M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/800 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/100 [00:00<?, ? examples/s]

Processing train split...
Processing validation split...
Processing test split...

✅ Готово! Обработано изображений: 1000


In [2]:
!zip -r /content/yolov5.zip /content/yolov5

  adding: content/yolov5/ (stored 0%)
  adding: content/yolov5/dataset/ (stored 0%)
  adding: content/yolov5/dataset/custom_train.txt (deflated 93%)
  adding: content/yolov5/dataset/custom_valid.txt (deflated 92%)
  adding: content/yolov5/dataset/custom_train_test.txt (deflated 92%)
  adding: content/yolov5/dataset/custom_data/ (stored 0%)
  adding: content/yolov5/dataset/custom_data/image111.jpg (deflated 6%)
  adding: content/yolov5/dataset/custom_data/image038.txt (deflated 51%)
  adding: content/yolov5/dataset/custom_data/image013.txt (deflated 71%)
  adding: content/yolov5/dataset/custom_data/image510.txt (deflated 56%)
  adding: content/yolov5/dataset/custom_data/image260.txt (deflated 58%)
  adding: content/yolov5/dataset/custom_data/image747.txt (deflated 57%)
  adding: content/yolov5/dataset/custom_data/image442.txt (deflated 65%)
  adding: content/yolov5/dataset/custom_data/image605.txt (deflated 66%)
  adding: content/yolov5/dataset/custom_data/image744.jpg (deflated 1%)
  a